In [1]:
import requests
import pandas as pd

url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&outputsize=full&apikey=WGZXNCUUJJGWK8QA"
r = requests.get(url)
data = r.json()
df=pd.DataFrame(data)
df1=pd.DataFrame()
df.reset_index(inplace=True)
compnay_name=df['Meta Data'][1]
df.drop(['Meta Data'],axis=1,inplace=True)
df.drop(index=df.index[0:5],inplace=True)   
df.reset_index(inplace=True,drop=True)
df1['Date']=df['index'].apply(lambda x: x)
df1['Company_Name']=compnay_name
df1['Open']=df['Time Series (Daily)'].apply(lambda x: x['1. open'])
df1['High']=df['Time Series (Daily)'].apply(lambda x: x['2. high']) 
df1['Low']=df['Time Series (Daily)'].apply(lambda x: x['3. low'])
df1['Close']=df['Time Series (Daily)'].apply(lambda x: x['4. close'])
df1['Volume']=df['Time Series (Daily)'].apply(lambda x: x['5. volume'])





6266

In [5]:
df1.to_csv('IBM.csv',index=False)

In [3]:
from calendar import c
import os
from uu import Error
from dotenv import load_dotenv
load_dotenv()
import json
import requests
import pandas as pd

def pre_process_data(company_name,):
    api_key = os.getenv("ALPHAVANTAGE_API_KEY")
    try:
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={company_name}&outputsize=full&apikey={api_key}"
        r = requests.get(url)
        data = r.json()
        df=pd.DataFrame(data)
        df1=pd.DataFrame()
        df.reset_index(inplace=True)
        compnay_name=df['Meta Data'][1]
        df.drop(['Meta Data'],axis=1,inplace=True)
        df.drop(index=df.index[0:5],inplace=True)   
        df.reset_index(inplace=True,drop=True)
        df1['Date']=df['index'].apply(lambda x: x)
        df1['Company_Name']=compnay_name
        df1['Open']=df['Time Series (Daily)'].apply(lambda x: x['1. open'])
        df1['High']=df['Time Series (Daily)'].apply(lambda x: x['2. high']) 
        df1['Low']=df['Time Series (Daily)'].apply(lambda x: x['3. low'])
        df1['Close']=df['Time Series (Daily)'].apply(lambda x: x['4. close'])
        df1['Volume']=df['Time Series (Daily)'].apply(lambda x: x['5. volume'])
        return df1, company_name
    except ValueError:
        # print("Error in fetching data, check the company name along with the API Key")
        return None, company_name 
def main():
    with open("company_list.json", "r") as f:
        data = json.load(f)
    for sector in data:
        company_list = data[sector]
        for company_name in company_list:
            data, company_name = pre_process_data(company_name)
            if data is not None:
               dict_name[company_name]=len(data) 
            else:
                 dict_name[company_name]=None
    print(dict_name)
        
if __name__ == "__main__":
    dict_name={}
    main()

Error fetching data for Alphabet (Google): If using all scalar values, you must pass an index
Error fetching data for AT&T: If using all scalar values, you must pass an index
Error fetching data for Verizon: If using all scalar values, you must pass an index
Error fetching data for Comcast: If using all scalar values, you must pass an index
Error fetching data for Walt Disney: If using all scalar values, you must pass an index
Error fetching data for Fox Corporation: If using all scalar values, you must pass an index
Error fetching data for T-Mobile US: If using all scalar values, you must pass an index
Error fetching data for Charter Communications: If using all scalar values, you must pass an index
Error fetching data for Liberty Global: If using all scalar values, you must pass an index
Error fetching data for Netflix: If using all scalar values, you must pass an index
Error fetching data for Spotify: If using all scalar values, you must pass an index
Error fetching data for Twitter

In [4]:
dict_name



{'Alphabet (Google)': None,
 'AT&T': None,
 'Verizon': None,
 'Comcast': None,
 'Walt Disney': None,
 'Fox Corporation': None,
 'T-Mobile US': None,
 'Charter Communications': None,
 'Liberty Global': None,
 'Netflix': None,
 'Spotify': None,
 'Twitter': None,
 'Snap Inc.': None,
 'Zoom Video Communications': None,
 'Facebook (Meta)': None,
 'Pinterest': None,
 'Reddit': None,
 'Vivendi': None,
 'WarnerMedia (now Warner Bros. Discovery)': None,
 'Dish Network': None,
 'Discovery Communications': None}

In [31]:
dict_name

{'IBM': None}